In [4]:
import requests
import json

In [91]:
match_id = 2853522349

In [92]:
host = "https://api.opendota.com/api/matches/" + str(match_id)
data = {'match_id': match_id}

In [93]:
request = requests.get(host, data=data)

In [94]:
request.status_code

200

In [95]:
match_data = request.content
match_data = json.loads(match_data)

In [96]:
match_data.keys()

[u'replay_url',
 u'barracks_status_dire',
 u'match_id',
 u'barracks_status_radiant',
 u'cluster',
 u'replay_salt',
 u'first_blood_time',
 u'chat',
 u'dire_score',
 u'duration',
 u'radiant_win',
 u'skill',
 u'lobby_type',
 u'tower_status_dire',
 u'teamfights',
 u'series_type',
 u'objectives',
 u'version',
 u'cosmetics',
 u'leagueid',
 u'engine',
 u'radiant_score',
 u'human_players',
 u'start_time',
 u'match_seq_num',
 u'series_id',
 u'tower_status_radiant',
 u'negative_votes',
 u'picks_bans',
 u'positive_votes',
 u'radiant_xp_adv',
 u'region',
 u'game_mode',
 u'patch',
 u'radiant_gold_adv',
 u'players']

Fields of interest in match:
    1. version AND patch
    2. positive_votes and negative_votes
    3. first_blood_time
    4. barracks_status_radiant
    5. barracks_status_dire
    6. radiant_win
    7. possibly teamfights in the future

In [97]:
# Here we analyze a player
player1 = match_data['players'][0]

In [98]:
player1.keys()

[u'gold',
 u'actions',
 u'damage_inflictor_received',
 u'lh_t',
 u'damage_taken',
 u'kill_streaks',
 u'cosmetics',
 u'hero_id',
 u'kills_log',
 u'account_id',
 u'kills',
 u'kills_per_min',
 u'start_time',
 u'isRadiant',
 u'backpack_1',
 u'backpack_0',
 u'leaver_status',
 u'killed',
 u'stuns',
 u'gold_per_min',
 u'name',
 u'level',
 u'damage_inflictor',
 u'patch',
 u'item_4',
 u'item_5',
 u'item_2',
 u'item_3',
 u'item_0',
 u'lose',
 u'sen_left_log',
 u'obs_placed',
 u'match_id',
 u'kda',
 u'pings',
 u'gold_reasons',
 u'total_gold',
 u'item_uses',
 u'duration',
 u'sen',
 u'lobby_type',
 u'denies',
 u'killed_by',
 u'permanent_buffs',
 u'last_login',
 u'gold_t',
 u'xp_per_min',
 u'ability_uses',
 u'item_1',
 u'abandons',
 u'rune_pickups',
 u'assists',
 u'obs_left_log',
 u'xp_reasons',
 u'purchase_log',
 u'region',
 u'obs_log',
 u'times',
 u'obs',
 u'win',
 u'radiant_win',
 u'runes',
 u'game_mode',
 u'ability_upgrades_arr',
 u'party_id',
 u'hero_healing',
 u'life_state',
 u'xp_t',
 u'bench

https://github.com/kronusme/dota2-api/blob/master/data/heroes.json
https://github.com/kronusme/dota2-api/blob/master/data/items.json

Of interest from the player fields are:
    1. gold
    2. last_hints
    3. damage_taken
    4. isRadiant
    5. hero_id
    6. actions_per_min
    7. kills
    8. kills_per_min
    9. item_win for end state items
    10. kda
    11. assists
    12. deaths
    13. total_gold and total_xp
    14. denies
    15. item_0, item_1, item_2, ..., item_5
    16. lane_role
    17. lane_efficiency_pct
    18. solo_competitive_rank
    19. hero_damage
    20. hero_healing

In [50]:
def extractDotaInformation(matchJSON):
    matchInfo = {}
    matchInfo['version'] = matchJSON['version']
    matchInfo['patch'] = matchJSON['patch']
    matchInfo['positive_votes'] = matchJSON['positive_votes']
    matchInfo['negative_votes'] = matchJSON['negative_votes']
    matchInfo['first_blood_time'] = matchJSON['first_blood_time']
    matchInfo['barracks_status_radiant'] = matchJSON['barracks_status_radiant']
    matchInfo['barracks_status_dire'] = matchJSON['barracks_status_dire']
    matchInfo['tower_status_radiant'] = matchJSON['tower_status_radiant']
    matchInfo['tower_status_dire'] = matchJSON['tower_status_dire']
    matchInfo['radiant_win'] = matchJSON['radiant_win']
    matchInfo['duration'] = matchJSON['duration']
    # player info now
    # organize the heroes into positions and radiant vs dire

    for player in matchJSON['players']:
        if player['isRadiant']:
            matchInfo['hero_radiant' + str(player['lane_role'])] = player['hero_id']
            print player['lane_role']
            
            # rad.remove(player['lane_role'])
        else:
            matchInfo['hero_dire' + str(player['lane_role'])] = player['hero_id']
            print player['lane_role']
            
            # dire.remove(player['lane_role'])
    if len(rad) != 0 and len(rad) != len(dire):
        raise Exception()
    
    

In [51]:
entriesPerFile = 2
f = open('data/test_matches_'+str(2853522349)+'.txt', 'wb')

for x in range(match_id, match_id + 10):
    host = "https://api.opendota.com/api/matches/" + str(x)
    data = {'match_id': x}
    response = requests.get(host, data=data)
    if response.status_code != 200:
        raise Exception("Status code is {0}".format(response.status_code))
    match = json.loads(response.content)
    relevantMatch = extractDotaInformation(match)
    f.write(json.dumps(relevantMatch) + '\n')

f.close()

[u'purchase_time', u'lane_efficiency', u'gold', u'sentry_kills', u'lane_efficiency_pct', u'actions', u'damage_inflictor_received', u'lh_t', u'damage_taken', u'kill_streaks', u'cosmetics', u'hero_id', u'necronomicon_kills', u'kills_log', u'sentry_uses', u'account_id', u'kills', u'kills_per_min', u'start_time', u'isRadiant', u'backpack_1', u'backpack_0', u'leaver_status', u'killed', u'stuns', u'gold_per_min', u'name', u'level', u'actions_per_min', u'damage_inflictor', u'life_state_dead', u'patch', u'item_4', u'item_5', u'item_2', u'item_3', u'item_0', u'lose', u'sen_left_log', u'buyback_count', u'obs_placed', u'match_id', u'kda', u'pings', u'gold_reasons', u'total_gold', u'item_uses', u'duration', u'sen', u'lobby_type', u'denies', u'ancient_kills', u'killed_by', u'neutral_kills', u'permanent_buffs', u'last_login', u'gold_t', u'xp_per_min', u'ability_uses', u'item_1', u'abandons', u'rune_pickups', u'assists', u'obs_left_log', u'xp_reasons', u'purchase_log', u'purchase_ward_sentry', u'regi

KeyError: 'lane_role'

In [5]:
heroInfo = {
        "heroes":[
        {
            "name":"npc_dota_hero_antimage",
                "id":1
        },
        {
            "name":"npc_dota_hero_axe",
            "id":2
        },
        {
            "name":"npc_dota_hero_bane",
            "id":3
        },
        {
            "name":"npc_dota_hero_bloodseeker",
            "id":4
        },
        {
            "name":"npc_dota_hero_crystal_maiden",
            "id":5
        },
        {
            "name":"npc_dota_hero_drow_ranger",
            "id":6
        },
        {
            "name":"npc_dota_hero_earthshaker",
            "id":7
        },
        {
            "name":"npc_dota_hero_juggernaut",
            "id":8
        },
        {
            "name":"npc_dota_hero_mirana",
            "id":9
        },
        {
            "name":"npc_dota_hero_nevermore",
            "id":11
        },
        {
            "name":"npc_dota_hero_morphling",
            "id":10
        },
        {
            "name":"npc_dota_hero_phantom_lancer",
            "id":12
        },
        {
            "name":"npc_dota_hero_puck",
            "id":13
        },
        {
            "name":"npc_dota_hero_pudge",
            "id":14
        },
        {
            "name":"npc_dota_hero_razor",
            "id":15
        },
        {
            "name":"npc_dota_hero_sand_king",
            "id":16
        },
        {
            "name":"npc_dota_hero_storm_spirit",
            "id":17
        },
        {
            "name":"npc_dota_hero_sven",
            "id":18
        },
        {
            "name":"npc_dota_hero_tiny",
            "id":19
        },
        {
            "name":"npc_dota_hero_vengefulspirit",
            "id":20
        },
        {
            "name":"npc_dota_hero_windrunner",
            "id":21
        },
        {
            "name":"npc_dota_hero_zuus",
            "id":22
        },
        {
            "name":"npc_dota_hero_kunkka",
            "id":23
        },
        {
            "name":"npc_dota_hero_lina",
            "id":25
        },
        {
            "name":"npc_dota_hero_lich",
            "id":31
        },
        {
            "name":"npc_dota_hero_lion",
            "id":26
        },
        {
            "name":"npc_dota_hero_shadow_shaman",
            "id":27
        },
        {
            "name":"npc_dota_hero_slardar",
            "id":28
        },
        {
            "name":"npc_dota_hero_tidehunter",
            "id":29
        },
        {
            "name":"npc_dota_hero_witch_doctor",
            "id":30
        },
        {
            "name":"npc_dota_hero_riki",
            "id":32
        },
        {
            "name":"npc_dota_hero_enigma",
            "id":33
        },
        {
            "name":"npc_dota_hero_tinker",
            "id":34
        },
        {
            "name":"npc_dota_hero_sniper",
            "id":35
        },
        {
            "name":"npc_dota_hero_necrolyte",
            "id":36
        },
        {
            "name":"npc_dota_hero_warlock",
            "id":37
        },
        {
            "name":"npc_dota_hero_beastmaster",
            "id":38
        },
        {
            "name":"npc_dota_hero_queenofpain",
            "id":39
        },
        {
            "name":"npc_dota_hero_venomancer",
            "id":40
        },
        {
            "name":"npc_dota_hero_faceless_void",
            "id":41
        },
        {
            "name":"npc_dota_hero_skeleton_king",
            "id":42
        },
        {
            "name":"npc_dota_hero_death_prophet",
            "id":43
        },
        {
            "name":"npc_dota_hero_phantom_assassin",
            "id":44
        },
        {
            "name":"npc_dota_hero_pugna",
            "id":45
        },
        {
            "name":"npc_dota_hero_templar_assassin",
            "id":46
        },
        {
            "name":"npc_dota_hero_viper",
            "id":47
        },
        {
            "name":"npc_dota_hero_luna",
            "id":48
        },
        {
            "name":"npc_dota_hero_dragon_knight",
            "id":49
        },
        {
            "name":"npc_dota_hero_dazzle",
            "id":50
        },
        {
            "name":"npc_dota_hero_rattletrap",
            "id":51
        },
        {
            "name":"npc_dota_hero_leshrac",
            "id":52
        },
        {
            "name":"npc_dota_hero_furion",
            "id":53
        },
        {
            "name":"npc_dota_hero_life_stealer",
            "id":54
        },
        {
            "name":"npc_dota_hero_dark_seer",
            "id":55
        },
        {
            "name":"npc_dota_hero_clinkz",
            "id":56
        },
        {
            "name":"npc_dota_hero_omniknight",
            "id":57
        },
        {
            "name":"npc_dota_hero_enchantress",
            "id":58
        },
        {
            "name":"npc_dota_hero_huskar",
            "id":59
        },
        {
            "name":"npc_dota_hero_night_stalker",
            "id":60
        },
        {
            "name":"npc_dota_hero_broodmother",
            "id":61
        },
        {
            "name":"npc_dota_hero_bounty_hunter",
            "id":62
        },
        {
            "name":"npc_dota_hero_weaver",
            "id":63
        },
        {
            "name":"npc_dota_hero_jakiro",
            "id":64
        },
        {
            "name":"npc_dota_hero_batrider",
            "id":65
        },
        {
            "name":"npc_dota_hero_chen",
            "id":66
        },
        {
            "name":"npc_dota_hero_spectre",
            "id":67
        },
        {
            "name":"npc_dota_hero_doom_bringer",
            "id":69
        },
        {
            "name":"npc_dota_hero_ancient_apparition",
            "id":68
        },
        {
            "name":"npc_dota_hero_ursa",
            "id":70
        },
        {
            "name":"npc_dota_hero_spirit_breaker",
            "id":71
        },
        {
            "name":"npc_dota_hero_gyrocopter",
            "id":72
        },
        {
            "name":"npc_dota_hero_alchemist",
            "id":73
        },
        {
            "name":"npc_dota_hero_invoker",
            "id":74
        },
        {
            "name":"npc_dota_hero_silencer",
            "id":75
        },
        {
            "name":"npc_dota_hero_obsidian_destroyer",
            "id":76
        },
        {
            "name":"npc_dota_hero_lycan",
            "id":77
        },
        {
            "name":"npc_dota_hero_brewmaster",
            "id":78
        },
        {
            "name":"npc_dota_hero_shadow_demon",
            "id":79
        },
        {
            "name":"npc_dota_hero_lone_druid",
            "id":80
        },
        {
            "name":"npc_dota_hero_chaos_knight",
            "id":81
        },
        {
            "name":"npc_dota_hero_meepo",
            "id":82
        },
        {
            "name":"npc_dota_hero_treant",
            "id":83
        },
        {
            "name":"npc_dota_hero_ogre_magi",
            "id":84
        },
        {
            "name":"npc_dota_hero_undying",
            "id":85
        },
        {
            "name":"npc_dota_hero_rubick",
            "id":86
        },
        {
            "name":"npc_dota_hero_disruptor",
            "id":87
        },
        {
            "name":"npc_dota_hero_nyx_assassin",
            "id":88
        },
        {
            "name":"npc_dota_hero_naga_siren",
            "id":89
        },
        {
            "name":"npc_dota_hero_keeper_of_the_light",
            "id":90
        },
        {
            "name":"npc_dota_hero_wisp",
            "id":91
        },
        {
            "name":"npc_dota_hero_visage",
            "id":92
        },
        {
            "name":"npc_dota_hero_slark",
            "id":93
        },
        {
            "name":"npc_dota_hero_medusa",
            "id":94
        },
        {
            "name":"npc_dota_hero_troll_warlord",
            "id":95
        },
        {
            "name":"npc_dota_hero_centaur",
            "id":96
        },
        {
            "name":"npc_dota_hero_magnataur",
            "id":97
        },
        {
            "name":"npc_dota_hero_shredder",
            "id":98
        },
        {
            "name":"npc_dota_hero_bristleback",
            "id":99
        },
        {
            "name":"npc_dota_hero_tusk",
            "id":100
        },
        {
            "name":"npc_dota_hero_skywrath_mage",
            "id":101
        },
        {
            "name":"npc_dota_hero_abaddon",
            "id":102
        },
        {
            "name":"npc_dota_hero_elder_titan",
            "id":103
        },
        {
            "name":"npc_dota_hero_legion_commander",
            "id":104
        },
        {
            "name":"npc_dota_hero_ember_spirit",
            "id":106
        },
        {
            "name":"npc_dota_hero_earth_spirit",
            "id":107
        },
        {
            "name":"npc_dota_hero_terrorblade",
            "id":109
        },
        {
            "name":"npc_dota_hero_phoenix",
            "id":110
        },
        {
            "name":"npc_dota_hero_oracle",
            "id":111
        },
        {
            "name":"npc_dota_hero_techies",
            "id":105
        },
        {
            "name":"npc_dota_hero_winter_wyvern",
            "id":112
        },
        {
            "name":"npc_dota_hero_arc_warden",
            "id":113
        },
        {
            "name":"npc_dota_hero_abyssal_underlord",
            "id":108
        },
        {
            "name":"npc_dota_hero_monkey_king",
            "id":114
        }
        ]
}

heroRoles = {
    1: [1], 2: [3, 2, 4], 3: [5, 4],
    4: [3, 2], 5: [5, 4], 6: [1, 2],
    7: [4, 5], 8: [1, 2], 9: [2, 3, 1],
    10: [1, 2], 11: [2, 1], 12: [1],
    13: [2], 14: [2], 15: [2],
    16: [3, 4], 17: [2], 18: [1],
    19: [2, 4], 20: [5, 4, 1], 21: [2],
    22: [2], 23: [2, 3], 25: [2, 4],
    26: [4, 5], 27: [5, 4, 2], 28: [4, 3, 1],
    29: [3, 4], 30: [4, 5], 31: [5],
    32: [4], 33: [4], 34: [2],
    35: [2, 1, 3], 36: [3, 4, 2], 37: [4, 5],
    38: [3, 4], 39: [2], 40: [4, 2],
    41: [3, 1], 42: [3, 1], 43: [2],
    44: [1], 45: [4, 2, 5], 46: [2],
    47: [2], 48: [1], 49: [2, 3, 1],
    50: [5, 4], 51: [3], 52: [2, 4],
    53: [4], 54: [4, 3], 55: [3],
    56: [3], 57: [5], 58: [4],
    59: [2, 1, 3], 60: [4, 2], 61: [3],
    62: [4], 63: [3], 64: [5, 2],
    65: [2, 3], 66: [4], 67: [1],
    68: [5], 69: [4, 3], 70: [4],
    71: [4, 3], 72: [1], 73: [2, 4],
    74: [2], 75: [5], 76: [2],
    77: [4], 78: [2], 79: [5],
    80: [4, 3], 81: [1, 4], 82: [2],
    83: [5], 84: [5], 85: [5, 4],
    86: [5], 87: [5], 88: [3],
    89: [1, 4], 90: [5], 91: [5],
    92: [4], 93: [1], 94: [1], 
    95: [1], 96: [3], 97: [2],
    98: [3, 2], 99: [3], 100: [3],
    101: [5, 4], 102: [5], 103: [3],
    104: [4], 105: [4], 106: [2],
    107: [4], 108: [3], 109: [1],
    110: [4], 111: [5], 112: [5],
    113: [2], 114: [2, 4]   
}
len(heroRoles)

113

In [ ]:
# return the roles of farm priority (1-5) for the corresponding hero ID
# the roles are in descending order of common if there is more than one
def classifyHeroPriority(heroID):
    